In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U transformers
!pip install -q -U peft
!pip install -q -U accelerate
!pip install -q -U datasets
!pip install -q -U trl

In [ ]:
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import os
import torch
from time import time
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    AutoModelForSequenceClassification,
    Trainer,
    EarlyStoppingCallback
)
from trl import SFTTrainer,setup_chat_format
import numpy as np
from transformers import BitsAndBytesConfig, AutoModelForSequenceClassification
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoTokenizer
from datasets import DatasetDict, Dataset

In [ ]:
class CFG:
    VER = 1
    NUM_LABELS = 3
    BATCH_SIZE = 4
    EPOCHS = 1
    
    MODEL_NAME = '/kaggle/input/llama-3/transformers/8b-chat-hf/1'

    
    SEED = 2024 
    MAX_LENGTH = 1024 
    
    OUTPUT_DIR = 'Llama 3 8b fine-tuned model'

# Prepare Dataset

In [ ]:
train = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/train.csv')
train = train[:int(len(train)*0.01)]
def process(input_str):
    stripped_str = input_str.strip('[]')
    sentences = [s.strip('"') for s in stripped_str.split('","')]
    return  ' '.join(sentences)

train.loc[:, 'prompt'] = train['prompt'].apply(process)
train.loc[:, 'response_a'] = train['response_a'].apply(process)
train.loc[:, 'response_b'] = train['response_b'].apply(process)

# Drop 'Null' for training
indexes = train[(train.response_a == 'null') & (train.response_b == 'null')].index
train.drop(indexes, inplace=True)
train.reset_index(inplace=True, drop=True)
train.loc[:, 'label'] = np.argmax(train[['winner_model_a','winner_model_b','winner_tie']].values, axis=1)

print(f"Total {len(indexes)} Null response rows dropped")
print('Total train samples: ', len(train))

train['text'] = 'User prompt: ' + train['prompt'] +  '\n\nModel A :\n' + train['response_a'] +'\n\n--------\n\nModel B:\n'  + train['response_b']
print(train['text'][4])

In [ ]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(train, test_size=0.2, random_state=42)

dataset_train = Dataset.from_pandas(train_df)
dataset_val = Dataset.from_pandas(val_df)

# Combine them into a single DatasetDict
dataset = DatasetDict({
    'train': dataset_train,
    'val': dataset_val,
})

# Model Loading -Quantization

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True, 
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_use_double_quant = True, 
    bnb_4bit_compute_dtype = torch.bfloat16 
)


model = AutoModelForSequenceClassification.from_pretrained(
    CFG.MODEL_NAME,
    quantization_config=quantization_config,
    num_labels=CFG.NUM_LABELS,
    device_map='auto'
)

# LoRA Configuration

In [ ]:
lora_config = LoraConfig(
    r = 16, 
    lora_alpha = 8,
    target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj'],
    lora_dropout = 0.05, 
    bias = 'none',
    task_type = 'SEQ_CLS'
)
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(CFG.MODEL_NAME, add_prefix_space=True)

tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'
tokenizer.add_eos_token = True

In [ ]:
model.config.pad_token_id = tokenizer.pad_token_id
# model.config.use_cache = False
model.config.pretraining_tp = 1

In [ ]:
def data_preprocesing(row):
    return tokenizer(row['text'], padding='max_length', truncation=True, max_length=CFG.MAX_LENGTH, return_tensors='np')

# dataset = Dataset.from_pandas(train)
tokenized_data = dataset.map(data_preprocesing, batched=True, remove_columns=['text'])
tokenized_data.set_format("torch")

# Finetune Llama 3: Model Training

In [ ]:
training_args = TrainingArguments(
    per_device_train_batch_size=CFG.BATCH_SIZE,
    num_train_epochs=CFG.EPOCHS,
    logging_dir = f'./logs_v{CFG.VER}',
    output_dir = f'./output_v{CFG.VER}',
    logging_steps=10,
    save_steps=10,
    logging_first_step=True,
    overwrite_output_dir=True,
    warmup_ratio=0.0,
    learning_rate=5e-5,
    lr_scheduler_type='constant',
    weight_decay=0.01,
    eval_steps=10,
    evaluation_strategy='steps',
    save_total_limit=2,
    report_to='none',
    load_best_model_at_end = True
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data['train'],
    eval_dataset=tokenized_data['val'],
    callbacks=[EarlyStoppingCallback(early_stopping_patience=10)]
)

train_result = trainer.train()